# Sequential Flatten + Dense

## 00. Imports

In [10]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

In [3]:
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
trainX.shape

(60000, 28, 28)

In [9]:
flat = np.ravel(trainX).reshape(shape[0], -1)
flat

NameError: name 'shape' is not defined